# Pipeline de Traitement des Données - Démonstration

Ce notebook démontre l'utilisation de la pipeline de traitement des données pour le projet RAG.

**Objectifs:**
- Scanner et cataloguer les documents
- Nettoyer les textes
- Découper en chunks optimisés
- Générer les embeddings
- Analyser les statistiques et la qualité

**Organisation:**
- Le code de traitement est maintenant dans `src/data_processor.py`
- Ce notebook sert à la visualisation et l'analyse

## 0. Setup et imports

In [ ]:
from pathlib import Path
import sys
import json
import pandas as pd
import numpy as np

# Configuration
ROOT = Path("..").resolve()
sys.path.insert(0, str(ROOT))

from src.data_processor import DataProcessor

print(f"Workspace: {ROOT}")

Workspace: H:\Documents\cyTech\Ing3\NLP\projet-nlp-rag


## 1. Initialisation de la pipeline

In [2]:
# Créer le processeur de données
data_dir = ROOT / "data"
processor = DataProcessor(data_dir)

print("Processeur initialisé")
print(f"  - Répertoire data: {processor.data_dir}")
print(f"  - Répertoire raw: {processor.raw_dir}")
print(f"  - Répertoire clean: {processor.clean_dir}")

Processeur initialisé
  - Répertoire data: H:\Documents\cyTech\Ing3\NLP\projet-nlp-rag\data
  - Répertoire raw: H:\Documents\cyTech\Ing3\NLP\projet-nlp-rag\data\raw
  - Répertoire clean: H:\Documents\cyTech\Ing3\NLP\projet-nlp-rag\data\clean


## 2. Scan et catalogage des documents

In [3]:
# Scanner les documents dans raw/
df_docs = processor.scan_documents(verbose=True)

# Afficher le catalogue
print(f"\n{'='*70}")
print(f"Catalogue des documents")
print(f"{'='*70}\n")
display(df_docs)

doc001: Armistice à Bordeaux | Source: Wikisource | Langue: fr
doc002: Convention d’armistice franco-allemande | Source: Wikisource | Langue: fr
doc003: Déclaration interalliée du 17 décembre 1942 | Source: Wikisource | Langue: fr
doc004: Le racisme hitlérien, machine de guerre contre la France | Source: Wikisource | Langue: fr
doc005: Les services statistiques français pendant l’Occupation | Source: Wikisource | Langue: fr
doc006: Témoignage (Lebrun) | Source: Wikisource | Langue: fr

Total: 6 documents
Sauvegardé: H:\Documents\cyTech\Ing3\NLP\projet-nlp-rag\data\docs.csv

Catalogue des documents



,doc_id,title,date,author,source,url,local_path,language
0,doc001,Armistice à Bordeaux,,,Wikisource,,H:/Documents/cyTech/Ing3/NLP/projet-nlp-rag/da...,fr
1,doc002,Convention d’armistice franco-allemande,,,Wikisource,,H:/Documents/cyTech/Ing3/NLP/projet-nlp-rag/da...,fr
2,doc003,Déclaration interalliée du 17 décembre 1942,,,Wikisource,,H:/Documents/cyTech/Ing3/NLP/projet-nlp-rag/da...,fr
3,doc004,"Le racisme hitlérien, machine de guerre contre...",,,Wikisource,,H:/Documents/cyTech/Ing3/NLP/projet-nlp-rag/da...,fr
4,doc005,Les services statistiques français pendant l’O...,,,Wikisource,,H:/Documents/cyTech/Ing3/NLP/projet-nlp-rag/da...,fr
5,doc006,Témoignage (Lebrun),,,Wikisource,,H:/Documents/cyTech/Ing3/NLP/projet-nlp-rag/da...,fr


## 3. Nettoyage des documents

In [4]:
# Nettoyer tous les documents
cleaned_count = processor.clean_documents(verbose=True)

print(f"\n{cleaned_count} documents nettoyés")

Armistice_à_Bordeaux.txt -> Armistice_a_Bordeaux.txt
Convention_d’armistice_franco-allemande.txt -> Convention_darmistice_franco-allemande.txt
Déclaration_interalliée_du_17_décembre_1942.txt -> Declaration_interalliee_du_17_decembre_1942.txt
Le_racisme_hitlérien,_machine_de_guerre_contre_la_France.txt -> Le_racisme_hitlerien_machine_de_guerre_contre_la_France.txt
Les_services_statistiques_français_pendant_l’Occupation.txt -> Les_services_statistiques_francais_pendant_lOccupation.txt
Témoignage_(Lebrun).txt -> Temoignage_Lebrun.txt

Cleaned files in: H:\Documents\cyTech\Ing3\NLP\projet-nlp-rag\data\clean

6 documents nettoyés


## 4. Génération des chunks

**Paramètres de chunking:**
- `chunk_size`: 2000 caractères (~300-500 tokens)
- `overlap`: 200 caractères (~30-50 tokens)

**Stratégie:** Découpe intelligente par paragraphe > phrase > ligne

In [5]:
# Générer les chunks
df_chunks = processor.generate_chunks(
    chunk_size=2000,
    overlap=200,
    verbose=True
)

print(f"\nTotal de chunks générés: {len(df_chunks)}")


Total chunks: 1472
Wrote: H:\Documents\cyTech\Ing3\NLP\projet-nlp-rag\data\chunks.jsonl

Total de chunks générés: 1472


## 5. Analyse statistique des chunks

In [6]:
# Ajouter une colonne pour la longueur
df_chunks["len_chars"] = df_chunks["text"].str.len()
df_chunks["len_words"] = df_chunks["text"].str.split().str.len()
df_chunks["len_tokens_est"] = (df_chunks["len_chars"] / 4).astype(int)

# Statistiques globales
print("=" * 70)
print("STATISTIQUES GLOBALES")
print("=" * 70)

nb_docs = df_chunks["doc_id"].nunique()
nb_chunks = len(df_chunks)

print(f"\nDocuments sources    : {nb_docs}")
print(f"Chunks générés       : {nb_chunks}")
print(f"Moyenne chunks/doc   : {nb_chunks / nb_docs:.1f}")

print("\n" + "-" * 70)
print("DISTRIBUTION DES LONGUEURS")
print("-" * 70)

stats_df = pd.DataFrame({
    "Métrique": ["Caractères", "Mots", "Tokens (estimé)"],
    "Moyenne": [
        f"{df_chunks['len_chars'].mean():.0f}",
        f"{df_chunks['len_words'].mean():.0f}",
        f"{df_chunks['len_tokens_est'].mean():.0f}"
    ],
    "Médiane": [
        f"{df_chunks['len_chars'].median():.0f}",
        f"{df_chunks['len_words'].median():.0f}",
        f"{df_chunks['len_tokens_est'].median():.0f}"
    ],
    "Min": [
        f"{df_chunks['len_chars'].min():.0f}",
        f"{df_chunks['len_words'].min():.0f}",
        f"{df_chunks['len_tokens_est'].min():.0f}"
    ],
    "Max": [
        f"{df_chunks['len_chars'].max():.0f}",
        f"{df_chunks['len_words'].max():.0f}",
        f"{df_chunks['len_tokens_est'].max():.0f}"
    ],
    "Écart-type": [
        f"{df_chunks['len_chars'].std():.0f}",
        f"{df_chunks['len_words'].std():.0f}",
        f"{df_chunks['len_tokens_est'].std():.0f}"
    ]
})

display(stats_df)

STATISTIQUES GLOBALES

Documents sources    : 6
Chunks générés       : 1472
Moyenne chunks/doc   : 245.3

----------------------------------------------------------------------
DISTRIBUTION DES LONGUEURS
----------------------------------------------------------------------


,Métrique,Moyenne,Médiane,Min,Max,Écart-type
0,Caractères,677,161,1,1999,787
1,Mots,111,28,1,365,128
2,Tokens (estimé),169,40,0,499,197


## 6. Répartition par document

In [8]:
# Tableau détaillé par document
print("-" * 70)
print("RÉPARTITION DÉTAILLÉE PAR DOCUMENT")
print("-" * 70)

doc_stats = []
for doc_id in df_chunks["doc_id"].unique():
    doc_data = df_chunks[df_chunks["doc_id"] == doc_id]
    title = doc_data["title"].iloc[0] if pd.notna(doc_data["title"].iloc[0]) else doc_id
    
    doc_stats.append({
        "doc_id": doc_id,
        "titre": title[:40] + "..." if len(str(title)) > 40 else title,
        "nb_chunks": len(doc_data),
        "chars_moy": f"{doc_data['len_chars'].mean():.0f}",
        "chars_total": f"{doc_data['len_chars'].sum():.0f}",
        "tokens_moy": f"{doc_data['len_tokens_est'].mean():.0f}",
    })

df_doc_stats = pd.DataFrame(doc_stats)
display(df_doc_stats)

----------------------------------------------------------------------
RÉPARTITION DÉTAILLÉE PAR DOCUMENT
----------------------------------------------------------------------


,doc_id,titre,nb_chunks,chars_moy,chars_total,tokens_moy
0,doc001,Armistice à Bordeaux,209,221,46128,55
1,doc002,Convention d’armistice franco-allemande,156,218,34083,54
2,doc003,Déclaration interalliée du 17 décembre 1...,69,196,13515,49
3,doc004,"Le racisme hitlérien, machine de guerre ...",212,209,44299,52
4,doc005,Les services statistiques français penda...,284,834,236825,208
5,doc006,Témoignage (Lebrun),542,1147,621487,286


## 7. Exemples de chunks

In [9]:
# Afficher quelques exemples aléatoires
print("=" * 70)
print("EXEMPLES DE CHUNKS (vérification manuelle)")
print("=" * 70)

sample_chunks = df_chunks.sample(min(3, len(df_chunks)))

for idx, (_, chunk) in enumerate(sample_chunks.iterrows(), 1):
    print(f"\n{'─' * 70}")
    print(f"CHUNK #{idx}")
    print(f"{'─' * 70}")
    print(f"ID          : {chunk['chunk_id']}")
    print(f"Document    : {chunk['title'] if pd.notna(chunk['title']) and chunk['title'] else chunk['doc_id']}")
    print(f"Date        : {chunk['date'] if pd.notna(chunk['date']) and chunk['date'] else 'N/A'}")
    print(f"Source      : {chunk['source']}")
    print(f"Longueur    : {chunk['len_chars']} chars / {chunk['len_words']} mots / ~{chunk['len_tokens_est']} tokens")
    print(f"Position    : [{chunk['start_char']} - {chunk['end_char']}]")
    print(f"\nExtrait (200 premiers caractères):")
    print(f"{chunk['text'][:200]}...")

print(f"\n{'=' * 70}")

EXEMPLES DE CHUNKS (vérification manuelle)

──────────────────────────────────────────────────────────────────────
CHUNK #1
──────────────────────────────────────────────────────────────────────
ID          : doc001_0094
Document    : Armistice à Bordeaux
Date        : N/A
Source      : Wikisource
Longueur    : 119 chars / 20 mots / ~29 tokens
Position    : [23556 - 23677]

Extrait (200 premiers caractères):
d, et dont les oreilles perçoivent en orgue et en tonnerre chaque syllabe de l’hymne que récitent mes lèvres immobiles....

──────────────────────────────────────────────────────────────────────
CHUNK #2
──────────────────────────────────────────────────────────────────────
ID          : doc006_0166
Document    : Témoignage (Lebrun)
Date        : N/A
Source      : Wikisource
Longueur    : 1632 chars / 257 mots / ~408 tokens
Position    : [258362 - 259996]

Extrait (200 premiers caractères):
ssaire, oui, mais dans le cadre général de la République démocratique.

Ce qui a compliqué l

## 8. Génération des embeddings

**Modèle:** `sentence-transformers/all-MiniLM-L6-v2`
- Dimension: 384
- Rapide et performant pour la plupart des tâches
- Normalisé pour similarité cosinus

In [10]:
# Générer les embeddings
embeddings, chunk_ids = processor.generate_embeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    batch_size=64,
    verbose=True
)

print(f"\nInformations sur les embeddings:")
print(f"  - Nombre de vecteurs: {embeddings.shape[0]}")
print(f"  - Dimension: {embeddings.shape[1]}")
print(f"  - Type: {embeddings.dtype}")
print(f"  - Taille mémoire: {embeddings.nbytes / 1024 / 1024:.2f} MB")

Génération des embeddings pour 1472 chunks...


ModuleNotFoundError: No module named 'sentence_transformers'

## 9. Vérifications de qualité

In [ ]:
# Vérifications finales
print("=" * 70)
print("CHECKLIST DE QUALITÉ")
print("=" * 70)

checks = []

# 1. Fichiers générés
checks.append(("docs.csv existe", processor.docs_csv.exists()))
checks.append(("chunks.jsonl existe", processor.chunks_jsonl.exists()))
checks.append(("embeddings.npy existe", processor.embeddings_npy.exists()))
checks.append(("chunk_ids.npy existe", processor.chunk_ids_npy.exists()))

# 2. Cohérence des données
checks.append(("Nombre chunks = nombre embeddings", len(df_chunks) == len(embeddings)))
checks.append(("Nombre chunk_ids = nombre embeddings", len(chunk_ids) == len(embeddings)))

# 3. Qualité des chunks
empty_chunks = (df_chunks["text"].str.strip().str.len() == 0).sum()
checks.append(("Pas de chunks vides", empty_chunks == 0))

missing_sources = (df_chunks["source"].fillna("") == "").sum()
checks.append((f"Sources renseignées ({len(df_chunks) - missing_sources}/{len(df_chunks)})", missing_sources == 0))

# 4. Embeddings
embeddings_normalized = np.allclose(np.linalg.norm(embeddings, axis=1), 1.0, atol=1e-5)
checks.append(("Embeddings normalisés (L2)", embeddings_normalized))

# Afficher les résultats
for check_name, passed in checks:
    status = "[OK]" if passed else "[FAIL]"
    print(f"{status} {check_name}")

print(f"\n{'=' * 70}")

all_passed = all(passed for _, passed in checks)
if all_passed:
    print("TOUS LES CONTRÔLES SONT PASSÉS !")
    print("Dataset prêt pour l'indexation et la recherche.")
else:
    print("ATTENTION: Certains contrôles ont échoué. Vérifier les détails ci-dessus.")

## Résumé de la pipeline

**Fichiers générés:**
1. `data/docs.csv` - Catalogue des documents sources
2. `data/chunks.jsonl` - Tous les chunks avec métadonnées
3. `data/embeddings.npy` - Vecteurs d'embeddings (normalisés)
4. `data/chunk_ids.npy` - IDs correspondants aux embeddings

**Prochaines étapes:**
1. Indexation avec FAISS (voir `src/index.py`)
2. Implémentation du retriever (voir `src/retriever.py`)
3. Intégration avec le LLM pour la génération
4. Évaluation et benchmarking

**Pour réexécuter la pipeline complète en ligne de commande:**
```bash
python -m src.scripts.build_data_pipeline --chunk-size 2000 --overlap 200
```